In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Entrenamiento del Modelo de Clasificación de Documentos"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "from src.models.cnn_model import DocumentClassifier\n",
    "from src.models.spark_processor import SparkImageProcessor\n",
    "import numpy as np\n",
    "from sklearn.model_selection import train_test_split\n",
    "import os\n",
    "import tensorflow as tf"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Cargar datos procesados\n",
    "processed_dir = '../data/processed'\n",
    "X = []\n",
    "y = []\n",
    "\n",
    "# Asumimos que los archivos están etiquetados en su nombre\n",
    "# ej: liquidacion_1.npy para positivos, other_1.npy para negativos\n",
    "for file in os.listdir(processed_dir):\n",
    "    if file.endswith('.npy'):\n",
    "        path = os.path.join(processed_dir, file)\n",
    "        img = np.load(path)\n",
    "        X.append(img)\n",
    "        y.append(1 if 'liquidacion' in file.lower() else 0)\n",
    "\n",
    "X = np.array(X)\n",
    "y = np.array(y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Dividir datos\n",
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    X, y, test_size=0.2, random_state=42\n",
    ")\n",
    "\n",
    "# Crear y entrenar modelo\n",
    "classifier = DocumentClassifier(input_shape=(224, 224, 1))\n",
    "classifier.compile_model()\n",
    "\n",
    "# Entrenar\n",
    "history = classifier.train(\n",
    "    train_data=(X_train, y_train),\n",
    "    validation_data=(X_test, y_test),\n",
    "    epochs=20,\n",
    "    batch_size=32,\n",
    "    model_save_path='../data/models/document_classifier.h5'\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Visualizar resultados del entrenamiento\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "plt.figure(figsize=(12, 4))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(history.history['loss'], label='Training Loss')\n",
    "plt.plot(history.history['val_loss'], label='Validation Loss')\n",
    "plt.title('Model Loss')\n",
    "plt.xlabel('Epoch')\n",
    "plt.ylabel('Loss')\n",
    "plt.legend()\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.plot(history.history['accuracy'], label='Training Accuracy')\n",
    "plt.plot(history.history['val_accuracy'], label='Validation Accuracy')\n",
    "plt.title('Model Accuracy')\n",
    "plt.xlabel('Epoch')\n",
    "plt.ylabel('Accuracy')\n",
    "plt.legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  }
 ]
}